In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re
 

In [2]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []
directt = []
strr = []
genre=[]



In [3]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,17500,100)

#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2023-04-15&runtime>0,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') != None  else np.nan
        time.append(runtime)
        
        type_ = store.p.find("span", class_ = 'genre').text.replace("\n", '') if store.p.find("span", class_ = 'genre') != None  else np.nan
        genre.append(type_)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") !=None else np.nan
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "****"
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text if len(value) > 0 else 0
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else '%^%^%^'
        gross.append(grosses)
        
        text=store.find("p",{"class":""}).get_text()
        text = re.sub('[(){}<>\n|]', '',text)

        if text.find("Stars:") == -1 and text.find("Star:") == -1:
            text=text + "Stars:None"
        if text.find("Director:") == -1 and text.find("Directors:") == -1:
            text = "Directors:None" + text
        text=re.split(';|\*|\n |Stars:|Star:|Director:|Directors:',text)    

        directors = text[1].strip(" ")
        stars = text[2]
        
        directt.append(directors)
        strr.append(stars)
        
        
        

        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        description.append(description_)
        


In [4]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year,"Genre" : genre, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Gross": gross, 
                           "Description": description, "Directors":directt,"Stars":strr  })
movie_list.head(5)



,Movie Name,Year of Release,Genre,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description,Directors,Stars
0,Air,(I) (2023),"Drama, Sport",111 min,7.5,73,"79,688",%^%^%^,Follows the history of shoe salesman Sonny Vac...,Ben Affleck,"Matt Damon, Jason Bateman, Ben Affleck, Chris ..."
1,The Super Mario Bros. Movie,(2023),"Animation, Adventure, Comedy",92 min,7.2,46,"122,325",%^%^%^,The story of The Super Mario Bros. on their jo...,"Aaron Horvath, Michael Jelenic, Pierre Leduc, ...","Chris Pratt, Anya Taylor-Joy, Charlie Day, Jac..."
2,Dungeons & Dragons: Honor Among Thieves,(2023),"Action, Adventure, Comedy",134 min,7.3,72,"109,921",%^%^%^,A charming thief and a band of unlikely advent...,"John Francis Daley, Jonathan Goldstein","Chris Pine, Michelle Rodriguez, Regé-Jean Page..."
3,Evil Dead Rise,(2023),Horror,96 min,6.8,69,"68,865",%^%^%^,A twisted tale of two estranged sisters whose ...,Lee Cronin,"Mirabai Pease, Richard Crouchley, Anna-Maree T..."
4,Sisu,(2022),"Action, War",91 min,7.0,70,"24,242",%^%^%^,When an ex-soldier who discovers gold in the L...,Jalmari Helander,"Jorma Tommila, Aksel Hennie, Jack Doolan, Mimo..."
